In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_lattice as tfl

In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [3]:
only_use_strong = 54

dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset, only_use_strong)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 6196746 points (56.847699%)


In [4]:
dataset = dataset.drop(['row'], axis=1)
columns = [x for x in columns if x != 'row']
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

In [5]:
quartiles = np.percentile(dataset['rising_idx'], np.linspace(0,99,8))
print(quartiles)
quartiles = np.percentile(dataset['first_val'], np.linspace(0,99,8))
print(quartiles)

[ 40. 102. 159. 206. 261. 321. 405. 629.]
[0.0000e+00 0.0000e+00 3.0000e+00 1.2000e+01 3.6000e+01 1.1200e+02
 4.8000e+02 1.7355e+04]


In [6]:
def buildLattice(num_keypoints, lattice_size, columns):
    # Use ParallelCombination helper layer to group togehter calibration layers
    # which have to be executed in paralel in order to be able to use Sequential
    # model. Alternatively you can use functional API.
    combined_calibrators = tfl.layers.ParallelCombination()

    # Configure calibration layers for every feature:

    for ft in columns:
        kpts = np.percentile(dataset[ft], np.linspace(0,99,num_keypoints))
        for i in range(1,len(kpts)):
            while kpts[i] <= kpts[i-1]:
                kpts[i] = kpts[i] + 1
        calibrator = tfl.layers.PWLCalibration(
            # Every PWLCalibration layer must have keypoints of piecewise linear
            # function specified. Easiest way to specify them is to uniformly cover
            # entire input range by using numpy.linspace().
            input_keypoints=kpts,
#             input_keypoints=np.linspace(dataset[ft].min(),
#                                       dataset[ft].max(),
#                                       num=num_keypoints),
        # You need to ensure that input keypoints have same dtype as layer input.
        # You can do it by setting dtype here or by providing keypoints in such
        # format which will be converted to deisred tf.dtype by default.
        dtype=tf.float32,
        # Output range must correspond to expected lattice input range.
        output_min=0.0,
        output_max=lattice_size - 1.0,
        monotonicity='increasing')
        combined_calibrators.append(calibrator)

    # Create Lattice layer to nonlineary fuse output of calibrators. Don't forget
    # to specify monotonicity 'increasing' for any dimension which calibrator is
    # monotonic regardless of monotonicity direction of calibrator. This includes
    # partial monotonicity of CategoricalCalibration layer.
    lattice = tfl.layers.Lattice(
      lattice_sizes=[lattice_size for _ in columns],
      monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
      output_min=dataset['delay'].min(),
      output_max=dataset['delay'].max())

    model = keras.models.Sequential()
    # We have just 2 layer as far as Sequential model is concerned.
    # PWLConcatenate layer takes care of grouping calibrators.
    model.add(combined_calibrators)
    model.add(lattice)
    model.compile(loss=keras.losses.mean_squared_error,
                optimizer=keras.optimizers.Adagrad(learning_rate=80.0))
    return model




In [7]:
def trainLattice(model, modelName, cols, epochs):
    
    features = train_dataset[cols].values.astype(np.float32)
    target = train_labels.values.astype(np.float32)

    checkpoint = ModelCheckpoint('./' + modelName, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.05,
            shuffle=False,
            callbacks=[checkpoint])
    return model

In [8]:
show_perf = True
load_prior_model = True
train_model = True
strong_data_sfx = "_strong" if (only_use_strong > 0) else ""



# Hypercube_16_2 = buildLattice(16,2,columns[:-1])
# if (load_prior_model): 
#     Hypercube_16_2.build((None,4,))
#     Hypercube_16_2.load_weights('./Hypercube_16_2' + strong_data_sfx) #load
# if (train_model): 
#     Hypercube_16_2 = trainLattice(Hypercube_16_2,'Hypercube_16_2' + strong_data_sfx,columns[:-1], 30) #train
#     Hypercube_16_2.save('./Hypercube_16_2' + strong_data_sfx)
# preds = Hypercube_16_2.predict(test_dataset.values.astype(np.float32))
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractLatticeWeights(Hypercube_16_2, 'Hypercube_16_2' + strong_data_sfx)
# print('--------------------------------')


Hypercube_8_4 = buildLattice(8,4,columns[:-1])
if (load_prior_model): 
    Hypercube_8_4.build((None,4,))
    Hypercube_8_4.load_weights('./Hypercube_8_4' + strong_data_sfx) #load
if (train_model): 
    Hypercube_8_4 = trainLattice(Hypercube_16_2,'Hypercube_8_4' + strong_data_sfx,columns[:-1], 30) #train
    Hypercube_8_4.save('./Hypercube_8_4' + strong_data_sfx)
preds = Hypercube_8_4.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_8_4, 'Hypercube_8_4' + strong_data_sfx)
print('--------------------------------')


Train on 3574936 samples, validate on 188155 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/30
3574936/3574936 [==============================] - 960s 269us/sample - loss: 433972.1551 - val_loss: 258578.4270
Epoch 2/30
3574936/3574936 [==============================] - 892s 249us/sample - loss: 252330.2396 - val_loss: 76685.7327
Epoch 3/30
3574936/3574936 [==============================] - 660s 185us/sample - loss: 227911.8262 - val_loss: 55139.8669
Epoch 4/30
3574936/3574936 [==============================] - 908s 254us/sample - loss: 209001.3135 - val_loss: 222679.6514
Epoch 5/30
3574936/3574936 [==============================] - 893s 250us/sample - loss: 194178.7305 - val_loss: 46571.2440
Epoch 6/30
3574936/3574936 [==============================] - 860s 241us/sample - loss: 183009.4599 - val_loss: 52384.

3574936/3574936 [==============================] - 489s 137us/sample - loss: 111259.3243 - val_loss: 48007.3089
Epoch 28/30
3574936/3574936 [==============================] - 490s 137us/sample - loss: 110671.2357 - val_loss: 30149.1325
Epoch 29/30
3574936/3574936 [==============================] - 489s 137us/sample - loss: 108628.9413 - val_loss: 44215.5985
Epoch 30/30
3574936/3574936 [==============================] - 489s 137us/sample - loss: 107511.0723 - val_loss: 44469.3270
MSE, MAE, r2: 44672.205543,174.931575,0.957360
--------------------------------
TBD :)
--------------------------------
Train on 3574936 samples, validate on 188155 samples
Epoch 1/30
3574936/3574936 [==============================] - 509s 142us/sample - loss: 106487.4271 - val_loss: 47695.4210
Epoch 2/30
3574936/3574936 [==============================] - 489s 137us/sample - loss: 105247.8367 - val_loss: 25526.3493
Epoch 3/30
3574936/3574936 [==============================] - 490s 137us/sample - loss: 103637.33

3574936/3574936 [==============================] - 493s 138us/sample - loss: 86276.8257 - val_loss: 59621.8285
Epoch 26/30
3574936/3574936 [==============================] - 492s 138us/sample - loss: 86201.6092 - val_loss: 56609.9027
Epoch 27/30
3574936/3574936 [==============================] - 492s 138us/sample - loss: 84773.2114 - val_loss: 27597.8248
Epoch 28/30
3574936/3574936 [==============================] - 493s 138us/sample - loss: 83945.4347 - val_loss: 51137.0456
Epoch 30/30
1737760/3574936 [=============>................] - ETA: 4:07 - loss: 82473.1917

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3528832/3574936 [============================>.] - ETA: 6s - loss: 82844.1573

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
preds = Hypercube_8_4.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)